# Data Mart configuration and usage with `ibm-ai-openscale` python package

<img src="https://console-lon02-black.cdn.stage1.s-bluemix.net/catalog/proxy-image/service-icon?url=https%3A%2F%2Faiopenscale-broker-ys1prod.stage1.mybluemix.net%2Fpublic%2Ffeatured.png" align="left" alt="banner" width="65">

This notebook shows hot to create a data mart for Watson Machine Learning deployments. It includes steps for performance, bias and quality monitor configurations.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Score models](#scoring)
- [5. Data Mart](#datamart)
- [6. Visualization](#visualization)

<a id="setup"></a>
## 1. Setup

### 1.1 Installation and authentication

In [ ]:
!pip install --upgrade ibm-ai-openscale

In [ ]:
!pip install --upgrade watson-machine-learning-client

### ACTION: Restart kernel to make sure installed versions are used.

Import and initiate.

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *

### ACTION: Get `data_mart_id` (GUID) and `apikey`

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

How to get DATA_MART_ID (this is AI OpenScale instance GUID)
```
bx resource service-instance ‘instance_name’
```


In [ ]:
aios_credentials = {
  "data_mart_id": "***",
  "apikey": "***", 
  "url": "https://api.aiopenscale.cloud.ibm.com"
}

In [ ]:
# The code was removed by Watson Studio for sharing.

### ACTION: Add your Watson Machine Learning credentials here

In [ ]:
wml_credentials =  {
  "apikey": "***",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "***",
  "instance_id": "***,
  "password": "***",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***"
}

In [ ]:
# The code was removed by Watson Studio for sharing.

### ACTION: Add your PostgreSQL credentials here

In [ ]:
postgres_credentials = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
client = APIClient(aios_credentials)

In [ ]:
client.version

Put your schema name here.

In [ ]:
schemaName = 'data_mart'

You can use the code below to create the schema. If the schema already exists skipp this cell.

In [ ]:
import psycopg2

hostname = postgres_credentials['uri'].split('@')[1].split(':')[0]
port = postgres_credentials['uri'].split('@')[1].split(':')[1].split('/')[0]
user = postgres_credentials['uri'].split('@')[0].split('//')[1].split(':')[0]
password = postgres_credentials['uri'].split('@')[0].split('//')[1].split(':')[1]
dbname = 'compose'

conn_string = "host=" + hostname + " port=" + port + " dbname=" + dbname + " user=" + user + " password=" + password
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
try:
    query = "drop schema " + schemaName + " cascade"
    cursor.execute(query)
except:
    pass
finally:    
    try:
        query = "create schema " + schemaName
        cursor.execute(query)
    finally:    
        conn.close()

### 1.2 DataMart setup

In [ ]:
client.data_mart.setup(postgres_credentials=postgres_credentials, schema=schemaName)

In [ ]:
data_mart_details = client.data_mart.get_details()

Data Mart with instance id name has been created.

<a id="binding"></a>

## 2. Bind machine learning engines

### 2.1 Bind Watson Machine Learning 

You can bind you Watson Machine Learning instance to previously created data mart.

In [ ]:
binding_uid = client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(wml_credentials))

In [ ]:
bindings_details = client.data_mart.bindings.get_details()

In [ ]:
client.data_mart.bindings.list()

### 2.2 Check available assets from binded services

#### List available assets

In [ ]:
client.data_mart.bindings.list_assets()

#### Get metadata of available assets

In [ ]:
assets_details = client.data_mart.bindings.get_asset_details()

In [ ]:
source_assets_uids = []

for x in client.data_mart.bindings.get_asset_details():
    source_assets_uids.append(x['source_uid'])

In [ ]:
print(str(source_assets_uids))

<a id="subscriptions"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

In [ ]:
for uid in source_assets_uids:
    sub = client.data_mart.subscriptions.add(WatsonMachineLearningAsset(uid))
    print("Adding subscription for: " + str(uid))

#### Get subscriptions list

In [ ]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

#### List subscriptions

In [ ]:
client.data_mart.subscriptions.list()

### 3.2 Enable payload logging and performance monitor for both subscribed assets

In [ ]:
for uid in subscriptions_uids:
    subscription = client.data_mart.subscriptions.get(uid)
    subscription.payload_logging.enable()
    subscription.performance_monitoring.enable()

#### Get details of enabled payload logging

In [ ]:
payload_logging_details = subscription.payload_logging.get_details()

#### Print schema of payload_logging table

In [ ]:
subscription.payload_logging.print_table_schema()

### 3.3 Enable Quality (Accuracy) monitor for best heart drug asset

#### Quality monitoring

Get the subscription ID for heart-drug model

In [ ]:
subscriptions_details = client.data_mart.subscriptions.get_details()
heart_drug_subscription = None

for s in subscriptions_details['subscriptions']:
    if s['entity']['asset']['name'] == 'drug-selection':
        heart_drug_subscription = s['metadata']['guid']


In [ ]:
subscription = client.data_mart.subscriptions.get(heart_drug_subscription)

### ACTION: Put your Spark Service credentials here

In [ ]:
spark_credentials = {
                  "tenant_id": "****",
                  "tenant_id_full": "***",
                  "cluster_master_url": "https://spark.bluemix.net",
                  "tenant_secret": "***",
                  "instance_id": "****",
                  "plan": "ibm.SparkService.PayGoPersonal"
}

In [ ]:
# The code was removed by Watson Studio for sharing.

#### Enable Accuracy monitor for selected subscription

In [ ]:
subscription.quality_monitoring.enable(evaluation_method='multiclass', threshold=0.8, min_records=5, spark_credentials=spark_credentials)

#### Send some feedback records to feedback store. 

Feedback records are used to evaluate our model.

In [ ]:
subscription.feedback_logging.store(
            [
                [20.0, 'M', 'HIGH', 'HIGH', 0.715337, 0.074773, 'drugB'],
                [38.0, 'F', 'HIGH', 'NORMAL', 0.868924, 0.061023, 'drugB'],
                [68.0, 'F', 'HIGH', 'NORMAL', 0.77541, 0.0761, 'drugB'],
                [65.0, 'M', 'HIGH', 'NORMAL', 0.635551, 0.056043, 'drugB'],
                [30.0, 'F', 'HIGH', 'HIGH', 0.800607, 0.060181, 'drugB'],
                [70.0, 'M', 'HIGH', 'HIGH', 0.658606, 0.047153, 'drugB'],
                [60.0, 'M', 'HIGH', 'HIGH', 0.805651, 0.057821, 'drugB'],
                [59.0, 'M', 'HIGH', 'HIGH', 0.816356, 0.058583, 'drugB'],
                [60.0, 'F', 'HIGH', 'HIGH', 0.800607, 0.060181, 'drugB'],
                [70.0, 'M', 'HIGH', 'HIGH', 0.658606, 0.047153, 'drugB'],
                [60.0, 'M', 'HIGH', 'HIGH', 0.805651, 0.057821, 'drugB'],
                [59.0, 'M', 'HIGH', 'HIGH', 0.816356, 0.058583, 'drugB']
            ],
            fields=['AGE', 'SEX', 'BP', 'CHOLESTEROL', 'NA', 'K', 'DRUG']
        )

<a id="scoring"></a>
## 4. Score models
Working with `watson-machine-learning-client`.

#### Get the initialized `watson-machine-learning` client

In [ ]:
wml_client = client.data_mart.bindings.get_native_engine_client(binding_uid=binding_uid)

`watson-machine-learning-client` documentation can be found [here](wml-api-pyclient.mybluemix.net).

Let's list our deployments.

In [ ]:
wml_client.deployments.list()

Let's extract scoring_ednpoints for both deployments.

In [ ]:
heart_scoring_endpoint = None
product_scoring_endpoint = None

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment['entity']['deployable_asset']['name'] == 'drug-selection':
      heart_scoring_endpoint = deployment['entity']['scoring_url']
    elif deployment['entity']['deployable_asset']['name'] == 'product-line-prediction':
      product_scoring_endpoint = deployment['entity']['scoring_url']

### 4.1 Score the best-heart drug model

In [ ]:
payload_scoring = {"fields": ["AGE", "SEX", "BP", "CHOLESTEROL","NA","K"],
                           "values": [[20.0, "F", "HIGH", "HIGH", 0.71, 0.07], [55.0, "M", "LOW", "HIGH", 0.71, 0.07]]}

for i in range(1,100):
    scores = wml_client.deployments.score(heart_scoring_endpoint, payload_scoring)

print(str(scores))


### 4.2 Score the product-line model

In [ ]:
payload_scoring = {"fields": ["GENDER", "AGE", "MARITAL_STATUS", "PROFESSION"],
                           "values": [["M", 23, "Single", "Student"], ["M", 55, "Single", "Executive"], ["F", 23, "Single", "Student"], ["F", 55, "Single", "Executive"]]}

for i in range(1,100):
    scores = wml_client.deployments.score(product_scoring_endpoint, payload_scoring)
    
print(str(scores))

<a id="datamart"></a>
## 5. Data mart

### 5.1 Accessing tables data via subscription
- `show_table`
- `describe_table`
- `get_table_content(format='pandas')`

#### Preview table content

In [ ]:
subscription.payload_logging.show_table()

In [ ]:
subscription.performance_monitoring.show_table()

In [ ]:
subscription.quality_monitoring.show_table()

#### Describe table (basic stats on table)

In [ ]:
subscription.payload_logging.describe_table()

#### Return table content as pandas dataframe

In [ ]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

In [ ]:
%matplotlib inline

pandas_df.boxplot()

### 5.2 Accessing data mart metrics
- performance
- quality
- fairness

Get all metrics.

In [ ]:
client.data_mart.get_deployment_metrics()

Get metric for particular deployment.

In [ ]:
subscription.quality_monitoring.get_metrics(deployment_uid=subscription.get_deployment_uids()[0])

<a id="setup"></a>
## 6. Visualization

Get the performance metrics as pandas dataframe.

In [ ]:
performance_pdf = subscription.performance_monitoring.get_table_content()

Let's enrich our data by calculating additional metric: `throughput`

In [ ]:
performance_pdf['throughput'] = performance_pdf.apply(lambda row: float((row.records/row.response_time)*1000) ,axis=1)

In [ ]:
performance_pdf

Let's plot `throughput` over time.

In [ ]:
performance_pdf.plot.line(y='throughput', title='Throughput (number of scored records per second)');

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.